<h1>MassBalanceMachine XGBoost Model Training - Iceland Region Example</h1>
<p style='text-align: justify;'>
In this notebook, we will simulate the glacier surface mass balance for the Iceland region using a custom <a href='https://xgboost.readthedocs.io/en/stable/'>XGBoost</a> model. The XGBoost model is designed with a custom objective function that generates monthly predictions based on aggregated observational data. We will create an instance of <code>CustomXGBoostRegressor</code> and train it using this custom loss function on the stake data from Iceland, which we have prepared in earlier notebooks. If you haven't already, please review the <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>data preprocessing</a> and <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>data processing WGMS</a> notebooks for more details.</p>

<p style='text-align: justify;'>
The workflow includes several key steps:
</p>

<ol style="margin-left: 20px; padding-left: 0;">
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Data Loading and Preparation:</strong> A <code>Dataloader</code> object is created to handle the loading of data and the creation of a training and testing split. This object also manages the generation of data splits for cross-validation.</p>
    </li>
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Cross-Validation and Model Training:</strong> Using Scikit-learn's cross-validation techniques, we explore different hyperparameters and train the model on the prepared data splits. This approach ensures a robust evaluation and helps in selecting suitable parameters.</p>
    </li>
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Aggregated Predictions:</strong> After training, we will display the aggregated monthly predictions generated by the model to visualize and analyze the results.</p>
    </li>
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Model Evaluation:</strong> Finally, the model's performance is evaluated on the test set, providing insights into its predictive accuracy for glacier mass balance.</p>
    </li>
</ol>

In [1]:
import pandas as pd
import massbalancemachine as mbm

In [2]:
# Set a random seed:
SEED = 42

In [3]:
data = pd.read_csv('./example_data/iceland/files/iceland_monthly_dataset.csv')
display(data)

,MONTHS,ID,RGIId,POINT_ID,YEAR,N_MONTHS,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,aspect,slope,t2m,tp,slhf,sshf,ssrd,fal,str
0,oct,0,RGI60-06.00228,hn14aa,1995,7,-18.773871,64.885013,2.07,1333.923612,-116.476388,1.606406,0.056246,267.885682,0.005071,-32688.346894,1.908546e+05,3.434260e+06,0.850005,-1.029337e+06
1,nov,0,RGI60-06.00228,hn14aa,1995,7,-18.773871,64.885013,2.07,1333.923612,-116.476388,1.606406,0.056246,266.376346,0.006053,301104.083653,8.280538e+05,8.424995e+05,0.849992,-1.431540e+06
2,dec,0,RGI60-06.00228,hn14aa,1995,7,-18.773871,64.885013,2.07,1333.923612,-116.476388,1.606406,0.056246,263.049011,0.005854,248241.745197,9.954409e+05,1.322171e+05,0.849992,-2.002829e+06
3,jan,0,RGI60-06.00228,hn14aa,1995,7,-18.773871,64.885013,2.07,1333.923612,-116.476388,1.606406,0.056246,261.692810,0.004156,348585.225978,1.243700e+06,4.884578e+05,0.849992,-1.792889e+06
4,feb,0,RGI60-06.00228,hn14aa,1995,7,-18.773871,64.885013,2.07,1333.923612,-116.476388,1.606406,0.056246,261.140088,0.002287,274514.643950,1.004845e+06,2.580602e+06,0.850005,-1.861757e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,jun,54,RGI60-06.00232,blt8,2018,11,-18.954942,64.662707,-2.63,1067.593874,246.593874,2.889730,0.018996,276.144439,0.003192,10994.304045,6.234279e+05,1.787212e+07,0.768536,-9.177811e+05
443,jul,54,RGI60-06.00232,blt8,2018,11,-18.954942,64.662707,-2.63,1067.593874,246.593874,2.889730,0.018996,277.553280,0.003422,-589325.605420,7.537638e+04,1.425122e+07,0.676787,-7.246837e+05
444,aug,54,RGI60-06.00232,blt8,2018,11,-18.954942,64.662707,-2.63,1067.593874,246.593874,2.889730,0.018996,276.193729,0.002495,-807105.778219,-1.563324e+05,1.332676e+07,0.677706,-1.884936e+06
445,sep,54,RGI60-06.00232,blt8,2018,11,-18.954942,64.662707,-2.63,1067.593874,246.593874,2.889730,0.018996,273.151130,0.003491,-432479.565272,1.434597e+05,9.078256e+06,0.717923,-2.152404e+06


<h2>1. Create the Train and Test Dataset and the Data Splits for Cross Validation</h2>
<p style='text-align: justify;'>
First, we create a <code>DataLoader</code> object, which generates both training and testing datasets, as well as the data splits required for cross-validation. To conserve memory, the <code>set_train_test_split</code> method returns iterators containing indices for the training and testing datasets. These indices are then used to retrieve the corresponding data for training and testing. Next, the <code>get_cv_split</code> method provides a list indicating the number of folds needed for cross-validation.
</p>


In [4]:
# Create a new DataLoader object with the monthly stake data measurements.
dataloader = mbm.DataLoader(data=data, random_seed=SEED)
# Create a training and testing iterators. The parameters are optional. The default value of test_size is 0.3.
train_itr, test_itr = dataloader.set_train_test_split(test_size=0.3,
                                                      shuffle=True)

# Get all indices of the training and testing dataset at once from the iterators. Once called, the iterators are empty.
train_indices, test_indices = list(train_itr), list(test_itr)

# Get the features and targets of the training data for the indices as defined above, that will be used during the cross validation.
df_X_train = data.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Create the cross validation splits based on the training dataset. The default value for the number of splits is 5.
splits = dataloader.get_cv_split(n_splits=5)

<h2>2. Create a CustomXGBoostRegressor Model</h2>
<p style='text-align: justify;'>
Next, we define the parameter ranges for each XGBoost parameter. In the subsequent step, we use cross-validation to explore these parameter ranges and select the combination that yields the lowest loss. Additionally, we create a <code>CustomXGBoostRegressor</code> object.
</p>

In [5]:
# For each of the XGBoost parameter, define the grid range
parameters = {
    'max_depth': [
        3,
        4,
        5,
        6,
    ],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 1]
}

In [6]:
# Create a CustomXGBoostRegressor instance
custom_xgboost = mbm.models.CustomXGBoostRegressor(device="cpu", random_seed=SEED)

<h2>3. Train the CustomXGBoostRegressor Model</h2>

<p style='text-align: justify; margin-bottom: 5px;'>
In the following cell, we begin training our model using either <strong>GridSearchCV</strong> or <strong>RandomizedSearchCV</strong>:
</p>

<ul style="margin-left: 20px; padding-left: 0; margin-bottom: 5px;">
  <li style="margin-bottom: 10px;">
    <p style='text-align: justify;'><strong>GridSearchCV</strong> performs an exhaustive search across all possible parameter combinations to find the best set for optimal performance using cross-validation. While this method is thorough, it is often time-consuming and computationally expensive.</p>
  </li>
  <li style="margin-bottom: 0px;">
    <p style='text-align: justify;'><strong>RandomizedSearchCV</strong>, on the other hand, samples a fixed number of parameter combinations from the distribution, making it more efficient in terms of time and computational resources, especially with larger hyperparameter spaces. However, this approach may miss some of the best parameter combinations that aren't selected in the random sampling.</p>
  </li>
</ul>

<p style='text-align: justify;'>
You can choose either of the two training methods. Both methods will use all CPU cores by default. If you want to adjust the number of cores used, you can change the <code>num_jobs</code> parameter.
</p>


In [11]:
# GridSearch
# custom_xgboost.gridsearch(parameters=parameters, splits=splits, features=df_X_train, targets=y_train, num_jobs=-1)

# RandomisedSearch, with n_iter the number of parameter settings that are sampled. Trade-off between goodness of the solution
# versus runtime.
custom_xgboost.randomsearch(parameters=parameters,
                            n_iter=20,
                            splits=splits,
                            features=df_X_train,
                            targets=y_train,
                            num_jobs=-1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/home/vmarijn/mambaforge/envs/MassBalanceMachine/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:43:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vmarijn/mambaforge/envs/MassBalanceMachine/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:43:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vmarijn/mambaforge/envs/MassBalanceMachine/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:43:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vmarijn/mambaforge/envs/MassBalanceMachine/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:43:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vmarijn/mambaforge

In [12]:
print("Best parameters:\n", custom_xgboost.param_search.best_params_)
print("Best score:\n", custom_xgboost.param_search.best_score_)

Best parameters:
 {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0}
Best score:
 -1.3838150292783424


<h4>3.1 Save the Trained Model</h4>

In [7]:
custom_xgboost.save_model('model.pkl')

In [8]:
# mbm.models.CustomXGBoostRegressor.load_model('model.pkl')

<h3>3.1 Show the Predictions per Fold</h3>